# Orbits of bitangents on symmetric quartics

## Table of contents
1. [Introduction](#introduction)
2. [Functions](#functions)
3. [Type I: The Klein ($\text{PSL}_2(7)$) quartic](#klein)
4. [Type II: The Clebsch](#fermat)
5. [Type III: The (48,33) quartic](#type3)
6. [Type IV: The $S_4$ quartic](#S4)
7. [Type V: The (16,13) quartic](#1613)
8. [Type VI: The $C_9$ quartic](#C9)
9. [Type VII: The $D_4$ quartic](#D4)
10. [Type IX: The $S_3$ quartic](#S3)
11. [Type X: The $K_4$ quartic](#K4)
12. [Type XI: The $C_3$ quartic](#C3)
13. [Type XII: The $C_2$ quartic](#C2)

# Introduction<a name="introduction"></a>
The following code is taken from [Vinzant's webpage](http://sites.math.washington.edu/~vinzant/research/quartics/quartictype.sage) from [her research on quartics](http://sites.math.washington.edu/~vinzant/research/quartics.html) with Plaumann and Sturmfels. It inputs a quartic polynomial and outputs (among other things) equations for its tritangents.

In [ ]:
%run Automorphism_Groups.ipynb

# Functions<a name="functions"></a>

In [ ]:
#INPUT: A homogeneous quartic in QQ[x,y,z]
#OUTPUT: The 28 bitangents (with floating point coefficients)
F=QQ; T.<x,y,z>=PolynomialRing(F,order='degrevlex')

def compute_bitangents(f):
    F=QQ; T.<x,y,z>=PolynomialRing(F)
    # Check whether the quartic is smooth
    Grad=ideal(f,diff(f,x),diff(f,y),diff(f,z))
    # if not Grad.dimension()==0:
    #     sys.exit("Quartic is not smooth!")
        
    R.<x,y,z,a,b,a0,a1,a2,a3,a4>=PolynomialRing(F)
    f0=f.base_extend(R)
    S.<a,b>=PolynomialRing(F)
    digits=50
    threshold=0.000000000001
    almostzero=threshold
    
    Line= a*x+b*y+z;
    puresquare=ideal(a0*a3^2-a1^2*a4,8*a0^2*a3-4*a0*a1*a2+a1^3,8*a1*a4^2-4*a2*a3*a4+a3^3,8*a0*a1*a4-4*a0*a2*a3+a1^2*a3,8*a0*a3*a4-4*a1*a2*a4+a1*a3^2,16*a0^2*a4+2*a0*a1*a3-4*a0*a2^2+a1^2*a2,16*a0*a4^2+2*a1*a3*a4-4*a2^2*a4+a2*a3^2);
    Res=f0.resultant(Line,z)    
    Res=Res.subs(y=1)    
    phi=hom(R,S,[0,0,0,a,b,Res.coefficient({x:0}),Res.coefficient({x:1}),Res.coefficient({x:2}),Res.coefficient({x:3}),Res.coefficient({x:4})])
    bit1 = phi(puresquare)     
    
    I_ideal=singular.groebner(singular(bit1))
    singular.lib('solve.lib')
    VRing=singular.solve(I_ideal,digits)
    singular.set_ring(VRing)
    B1=singular("SOL")
    
    nreal1=0
    Bitangents=[]
    RealBitangents=[]
    for k in [1..len(B1)]:
        real=0;
        if ((B1[k][1].impart()).absValue()<threshold) and ((B1[k][2].impart()).absValue()<threshold): 
            real=1
            RealBitangents=RealBitangents+[(float(B1[k][1].repart())+float(B1[k][1].impart())*i)*x+(float(B1[k][2].repart())+float(B1[k][2].impart())*i)*y+z]
        nreal1=nreal1+real
        Bitangents=Bitangents+[(float(B1[k][1].repart())+float(B1[k][1].impart())*i)*x+(float(B1[k][2].repart())+float(B1[k][2].impart())*i)*y+z]
    Line=a*x+y     
    Res=f0.resultant(Line,y)    
    Res=Res.subs(z=1)   
    phi=hom(R,S,[0,0,0,a,0,Res.coefficient({x:0}),Res.coefficient({x:1}),Res.coefficient({x:2}),Res.coefficient({x:3}),Res.coefficient({x:4})])  
    bit2=phi(puresquare)+ideal(b)
    
    if dimension(bit2)==-1: nreal2=0
    else: 
          I_ideal=singular.groebner(singular(bit2))
          singular.lib('solve.lib')
          VRing=singular.solve(I_ideal,digits)
          singular.set_ring(VRing)
          B2=singular("SOL")
          nreal2=0
          for k in [1..len(B2)]:
                real=0
                if ((B2[k][1].impart()).absValue()<threshold) and ((B2[k][2].impart()).absValue()<threshold):
                    real=1
                    RealBitangents=RealBitangents+[(float(B2[k][1].repart())+float(B2[k][1].impart())*i)*x+y]
                nreal2=nreal2+real  
                Bitangents=Bitangents+[(float(B2[k][1].repart())+float(B2[k][1].impart())*i)*x+y]
    
    Res=f0.resultant(x)
    Res=Res.subs(z=1)
    phi=hom(R,F,[0,0,0,0,0,Res.coefficient({y:0}),Res.coefficient({y:1}),Res.coefficient({y:2}),Res.coefficient({y:3}),Res.coefficient({y:4})])  
    bit3=phi(puresquare)
    if bit3==ideal(0):
        nreal3=1
        Bitangents=Bitangents+[x]
        RealBitangents=RealBitangents+[x]
    else: nreal3=0
    
    NRealBit=nreal1+nreal2+nreal3
    if len(Bitangents)!=28:
        return OSError("Something has gone wrong. We found "+str(len(Bitangents))+" bitangents")
    # print("The quartic has 28 bitangets, stored in 'Bitangents', and "+str(NRealBit)+" real bitangents, stored in 'RealBitangents'.")
    
    return Bitangents

In [ ]:
from sage.symbolic.expression_conversions import polynomial

# Round complex numbers
def round_cx(z0,n):
    return float(round(z0.real(),n)) + float(round(z0.imag(),n))*I

# The following inputs a line ax+by+cz and outputs the tuple of coefficients [a,b,c]
# Note that line.coefficients() has errors, in that a line line 3y+2z will output [3,2] rather than [0,3,2] which is what we want
def line_to_coeffs(bitangent):
    return [CC(bitangent.coefficient({x:1})), CC(bitangent.coefficient({y:1})),CC(bitangent.coefficient({z:1}))]

# Turn a list of three coefficients [a,b,c] into the polynomial ax+by+cz=0
def coeffs_to_line(list_of_coeffs):
    return x*list_of_coeffs[0] + y*list_of_coeffs[1] + z*list_of_coeffs[2]

# Provide a normalized version of the bitangent -- old (working) version
# def normalize(bitangent):
#     b = bitangent.change_ring(ComplexField(30))
#     if b.coefficients()[0] != 0:
#         c = b*(1/b.coefficients()[0])
#         if len(c.coefficients())==3:
#             new_coeffs = [round_cx(a,5) for a in c.coefficients()]
#             c = new_coeffs[0]*x + new_coeffs[1]*y + new_coeffs[2]*z
#         return c
#     return b



# Provide a normalized version of the bitangent
def normalize(bitangent):
    # Base change to CC
    b = bitangent.change_ring(ComplexField(30))

    # If the x-coordinate is nonzero mod out by it
    if line_to_coeffs(b)[0] != 0:
        c = b*(1/line_to_coeffs(b)[0])
    # Else mod out by the y-coordinate
    elif line_to_coeffs(b)[1] != 0:
        c = b*(1/line_to_coeffs(b)[1])
    # Or the z-coordinate
    else:
        c = b*(1/line_to_coeffs(b)[2])
    
    # Round the output coordinates and return a new line
    new_coeffs = [round_cx(a,4) for a in line_to_coeffs(c)]
    return coeffs_to_line(new_coeffs)


def compute_normalized_bitangents(f):
    return [normalize(b) for b in compute_bitangents(f)]

# Apply a matrix M to a line B = {ax+by+cy=0}
def gp_action(M,B):
    vec = matrix([[B.coefficient({x:1})],[B.coefficient({y:1})],[B.coefficient({z:1})]])
    output_mat = (M*vec).transpose()
    w = list(list(output_mat)[0])
    outputline = w[0]*x + w[1]*y + w[2]*z
    return normalize(outputline)

def extract_orbit(Bitangents,list_of_indices):
    orbit = [Bitangents[j] for j in list_of_indices]
    remaining_Bitangents = [b for b in Bitangents if b not in orbit]
    return orbit,remaining_Bitangents

In [ ]:
F=QQ; T.<x,y,z>=PolynomialRing(F)

bitan1 = 3*x + y + z
bitan2 = -0.5*y + z
bitan3 = -7*z


normalize(bitan3)

We write some functions to make the equations of bitangents more readable and act on them via a subgroup $G\le \text{PGL}_3(\mathbb{C})$.

In [ ]:
# We also add some commands we'll use throughout
def matrix_to_list(M):
    return [list(r) for r in M.rows()]

In [ ]:
# Generates a candidate list of the orbit of an input bitangent under a list of group elements
def candidate_orbit(bitangent,list_of_gp_elements):
    output_list = [bitangent]
    isotropy_gens = []
    for M in list_of_gp_elements:
        output_list.append(gp_action(M,bitangent))
        if gp_action(M,bitangent) == bitangent:
            isotropy_gens.append(M)
    for line in output_list:
        print(line)
    print('I think isotropy is generated by:\n')
    for isotropy_elt in isotropy_gens:
        print(isotropy_elt)

In [ ]:
def investigate_orbit(bitangent,group_dictionary):
    output_list = [bitangent]
    isotropy_gens = []
    # Iterate over the group
    for g in group_dictionary.keys():        
        output_list.append(gp_action(group_dictionary[g],bitangent))
        if gp_action(group_dictionary[g],bitangent) == bitangent:
            isotropy_gens.append(g)
    for line in set(output_list):
        print(line)
    print('I think isotropy is generated by:\n')
    for isotropy_elt in isotropy_gens:
        print(isotropy_elt)

In [ ]:
def trim(bitangent):
    x_coeff = CC(bitangent.coefficient({x:1}));
    y_coeff = CC(bitangent.coefficient({y:1}));
    z_coeff = CC(bitangent.coefficient({z:1}));
    return round_cx(x_coeff,3)*x + round_cx(y_coeff,3)*y + round_cx(z_coeff,3)*z

def investigate_orbit_Klein(bitangent,group_dictionary):
    output_list = [bitangent]
    isotropy_gens = []
    # Iterate over the group
    for g in group_dictionary.keys():        
        output_list.append(trim(gp_action(group_dictionary[g],bitangent)))
        if trim(gp_action(group_dictionary[g],bitangent)) == trim(bitangent):
            isotropy_gens.append([g,group_dictionary[g]])

    # Output the orbits
    for line in set(output_list):
        print(line)

    # Output the isotropy
    print('I think isotropy is generated by:\n')
    for isotropy_elt in isotropy_gens:
        print(isotropy_elt)

In [ ]:
############

def investigate_orbit_V(bitangent,group_dictionary):
    output_list = [bitangent]
    isotropy_gens = []
    # Iterate over the group
    for g in group_dictionary.keys():        
        output_list.append(gp_action(group_dictionary[g],bitangent))
        if gp_action(group_dictionary[g],bitangent) == bitangent:
            isotropy_gens.append([g,group_dictionary[g]])

    # Output the orbits
    for line in set(output_list):
        print(line)

    # Output the isotropy
    print('I think isotropy is generated by:\n')
    for isotropy_elt in isotropy_gens:
        print(isotropy_elt)
        # list_version = matrix_to_list(isotropy_elt)
        # # print(list_version)
        # output_list = []
        # for row in list_version:
        #     new_row = []
        #     for j in row:
        #         # every entry in the matx representation is either totally real or totally imaginary
        #         if j.real != 0:
        #             new_row.append(int(j.real()))
        #         elif j.imag != 0:
        #             new_row.append(int(j.imag())*I)
        #         else:
        #             print('error')
        #     output_list.append(new_row)
        # print(str(output_list) + ',')

# def investigate_orbit2(bitangent,group_dictionary):
#     output_list = [bitangent]
#     isotropy_gens = []
#     # Iterate over the group
#     for g in group_dictionary.keys():        
#         output_list.append(gp_action(group_dictionary[g],bitangent))
#         if gp_action(group_dictionary[g],bitangent) == bitangent:
#             isotropy_gens.append(group_dictionary[g])
#     for line in set(output_list):
#         print(line)
#     print('I think isotropy is generated by:\n')
#     for isotropy_elt in isotropy_gens:
#         print(isotropy_elt)

In [ ]:
def investigate_orbit_S4(bitangent,group_dictionary):
    output_list = [bitangent]
    isotropy_gens = []
    # Iterate over the group
    for g in group_dictionary.keys():        
        output_list.append(gp_action(group_dictionary[g],bitangent))
        if gp_action(group_dictionary[g],bitangent) == bitangent:
            isotropy_gens.append(group_dictionary[g])
    for line in set(output_list):
        print(line)
    print('I think isotropy is generated by:\n')
    for isotropy_elt in isotropy_gens:
        list_version = matrix_to_list(isotropy_elt)
        output_list = []
        for row in list_version:
            new_row = []
            for j in row:
                new_row.append(int(j.real()))
            output_list.append(new_row)
        print(str(output_list) + ',')

In [ ]:
def bitan_difference(bitan1,bitan2):
    # Take the difference in absolute value between each of the coefficients and sum these
    return abs(CC(bitan1.coefficient({x:1})) - CC(bitan2.coefficient({x:1}))) + abs(CC(bitan1.coefficient({y:1})) - CC(bitan2.coefficient({y:1}))) + abs(CC(bitan1.coefficient({z:1})) - CC(bitan2.coefficient({z:1})))

In [ ]:
bitan1 =x + (-1.1234 - 4.9221*I)*y + (-2.9252 - 1.4087*I)*z
bitan2 =x + (-1.1231 - 4.9225*I)*y + (-2.9254 - 1.4091*I)*z
# bitan_difference(bitan1,bitan2)
bitan_difference(bitan1,bitan2)

In [ ]:
def same_bitangent(bitan1,bitan2, error_rate=0.01):
    if bitan_difference(bitan1,bitan2) < error_rate:
        return true
    else:
        return false


In [ ]:
def investigate_orbit_NEW(bitangent,group_dictionary):
    output_list = [bitangent]

    for g in group_dictionary.keys():
        g_bitangent = gp_action(group_dictionary[g],bitangent)

        new_bitangent = True
        for b in output_list:
            if same_bitangent(g_bitangent,b):
                new_bitangent = False

        if new_bitangent == True:
            output_list.append(g_bitangent)
    return list(set(output_list))

def investigate_isotropy_NEW(bitangent,group_dictionary):
    isotropy_gens = []
    for g in group_dictionary.keys():
        # print(g)
        g_bitangent = gp_action(group_dictionary[g],bitangent)
        # print(g_bitangent)
        if same_bitangent(g_bitangent,bitangent):
            isotropy_gens.append(g)
    return isotropy_gens

We want to compute all the orbits of the bitangents for the [cubic surfaces found here](https://mat.uab.cat/~francesc/mates/autgen3.pdf).b

TODO -- FIX EVERYTHING ABOVE HERE/RENAME FUNCTIONS/ETC

# Type I: The Klein quartic<a name="klein"></a>
Our first quartic is the *Klein quartic*, defined by the equation
$$x^3y + y^3z + z^3x$$
This has the largest possible automorphism group, encoded as `PSL_dict` here.

It is a classical fact that the automorphism group of the Klein quartic acts transitively on its 28 bitangents, however we can also observe this computationally. We first compute a list of all the bitangents of the Klein quartic.

In [ ]:
Klein = x^3*y + y^3*z + z^3*x;
Klein_Bitangents = compute_normalized_bitangents(Klein);Klein_Bitangents

We can look at the first bitangent `Klein_Bitangents[0]`, and try to examine what its orbit is under the action of the automorphism group using our `investigate_orbit` command:

In [ ]:
investigate_orbit_NEW(Klein_Bitangents[0],PSL_dict)

It is clear to see that we get all 28 bitangents in the orbit of the first one under the action of $\text{PSL}_2(7)$. It remains to ask what the isotropy group of a single bitangent is --- since the action is transitive, we should obtain an index 48 subgroup in $\text{PSL}_2(7)$.

In [ ]:
investigate_isotropy_NEW(compute_normalized_bitangents(Klein)[0],PSL_dict)

We can then return to GAP and verify that these matrices generate a subgroup of $\text{PSL}_2(7)$ which is isomorphic to $S_3$. Therefore as a $\text{PSL}_2(7)$-set, the 28 bitangents of the Klein quartic are in equivariant bijection with the left cosets $[\text{PSL}_2(7)/S_3]$. As a sanity check, $|\text{PSL}_2(7)| = 168$ and $|S_3| = 6$, and $168/6 = 28$.

# Type II: Fermat<a href="fermat"></a>

## (II) Bitangents

In [ ]:
F=QQ; T.<x,y,z>=PolynomialRing(F)
Fermat = x^4 + y^4 + z^4

Fermat_Bitangents = compute_normalized_bitangents(Fermat);Fermat_Bitangents

Again, we can look at the orbit of the first bitangent:

In [ ]:
investigate_orbit_NEW(Fermat_Bitangents[0],Fermat_dict)

Hence we get 12 bitangents in this orbit. In terms of the indices of the list `Fermat_Bitangents`, these are the entries `[0,1,2,3,24,25,26,27,8,9,10,11]`. Again we can investigate the isotropy:

In [ ]:
investigate_isotropy_NEW(Fermat_Bitangents[0],Fermat_dict)

Going back to GAP, we get a copy of $C_8$ living inside the automorphism group of the Fermat quartic. Hence the 12 bitangents above correspond to the transitive set $[C_4^{\times 2}\rtimes S_3 / C_8]$.

We can look at the remaining bitangents, for instance the element `Fermat_Bitangents[4]` which was not in the orbit above:

In [ ]:
investigate_orbit_NEW(Fermat_Bitangents[4],Fermat_dict)

These are the remaining 16 bitangents -- their indices in `Fermat_Bitangents` are `[16,6,20,21,15,12,14,17,18,7,4,5,23,22,13,19]`. Again we study isotropy:

In [ ]:
investigate_isotropy_NEW(Fermat_Bitangents[4],Fermat_dict)

This gives us a copy of $C_6$ inside of $C_4^{\times 2}\rtimes S_3$. Hence altogether we get an equivariant bijection
$$\text{28 bitangen

# Type III: The $(48,33)$ quartic<a name="type3"></a>
There is a unique quartic of Type III, defined by the equation
$$x^4 + y^4 + z^4 + (4\rho+2)x^2y^2,$$
where $\rho$ is a primitive 3rd root of unity (note that $4\rho + 2 = 2\sqrt{-3}$, which is another common form for this quartic). Its automorphism group has order 48, and GAP ID `[48,33]`. It is accessible via `GpIII_Dict`.



## (III) Bitangents

We would like to compute equations for its bitangents, however since $\rho\notin\mathbb{Q}$, we must modify the function `compute_bitangents`. To that end we include the following

In [ ]:
def compute_bitangents_ideal_Q3(f):
    F=CyclotomicField(3); T.<x,y,z>=PolynomialRing(F)
    # Check whether the quartic is smooth
    # Grad=ideal(f,diff(f,x),diff(f,y),diff(f,z))
    # if not Grad.dimension()==0:
    #     sys.exit("Quartic is not smooth!")
    f = T(f)
    R.<x,y,z,a,b,a0,a1,a2,a3,a4>=PolynomialRing(F)
    f0=f.base_extend(R)
    S.<a,b>=PolynomialRing(F)
    digits=50
    threshold=0.000000000001
    almostzero=threshold

    # Make a formal line
    Line= a*x+b*y+z;
    
    puresquare=ideal(a0*a3^2-a1^2*a4,8*a0^2*a3-4*a0*a1*a2+a1^3,8*a1*a4^2-4*a2*a3*a4+a3^3,8*a0*a1*a4-4*a0*a2*a3+a1^2*a3,8*a0*a3*a4-4*a1*a2*a4+a1*a3^2,16*a0^2*a4+2*a0*a1*a3-4*a0*a2^2+a1^2*a2,16*a0*a4^2+2*a1*a3*a4-4*a2^2*a4+a2*a3^2);
    Res=f0.resultant(Line,z)    
    Res=Res.subs(y=1)    
    phi=hom(R,S,[0,0,0,a,b,Res.coefficient({x:0}),Res.coefficient({x:1}),Res.coefficient({x:2}),Res.coefficient({x:3}),Res.coefficient({x:4})])
    bit1 = phi(puresquare)    
    
    I_ideal=singular.groebner(singular(bit1))
    return I_ideal

Dolg = x^4 + y^4 + z^4 + 2*sqrt(-3)*(x^2)*(y^2)
F=CyclotomicField(12); T.<x,y,z>=PolynomialRing(F)

# Grad = ideal(Dolg,diff(Dolg,x),diff(Dolg,y),diff(Dolg,z))
Iideal = compute_bitangents_ideal_Q3(Dolg);Iideal
print(Iideal)

This gives us an ideal in a polynomial ring over $\mathbb{Q}(\zeta_3)$ that we wish to solve -- the solutions will be bitangents of the form $ax+by+z=0$.

In [ ]:
# Grad.dimension()
# cyc

R.<a,b> =PolynomialRing(CC)
rho = exp(2*pi*i/3)
a,b = var('a b')
f0 = a^4+(-4*rho-2)*a^2*b^2+b^4+4
f1 = 3*a^3*b^3+(2*rho+1)*a*b^5+(4*rho+2)*a*b
f2 = 7*a^2*b^5+(2*rho+1)*b^7+2*a^2*b+(8*rho+4)*b^3
f3 = 2*a*b^7+(2*rho+1)*a^3*b+a*b^3
f4 = 2*b^9-5*a^4*b+(-6*rho-3)*a^2*b^3+10*b^5+8*b


candidates = solve([f1==0,f2==0,f3==0,f0==0,f4==0],a,b); candidates

This gives us 24 bitangents. To obtain the other 4, we need to consider the case where $z=0$..... TODO

In [ ]:
QuarticIIIBis = [x + (-0.50000084 + 0.50000314*I)*z,
x + (-0.50000314 - 0.50000084*I)*z,
x - y + (0.500000000000000 - 0.866030000000000*I)*z,
x - y + (-0.500000000000000 + 0.866030000000000*I)*z,
x - y + (-0.866030000000000 - 0.500000000000000*I)*z,
x - y + (0.866030000000000 + 0.500000000000000*I)*z,
x + (0.50000314 + 0.50000084*I)*z,
x + (0.50000084 - 0.50000314*I)*z,
y + (0.50000314 + 0.50000084*I)*z,
y + (-0.50000084 + 0.50000314*I)*z,
y + (0.50000084 - 0.50000314*I)*z,
y + (-0.50000314 - 0.50000084*I)*z,
x + 1.00000000000000*I*y + (0.500000000000000 + 0.866030000000000*I)*z,
x + 1.00000000000000*I*y + (0.866030000000000 - 0.500000000000000*I)*z,
x + 1.00000000000000*I*y + (-0.866030000000000 + 0.500000000000000*I)*z,
x + 1.00000000000000*I*y + (-0.500000000000000 - 0.866030000000000*I)*z,
x + (-1.00000000000000*I)*y + (0.500000000000000 + 0.866030000000000*I)*z,
x + (-1.00000000000000*I)*y + (0.866030000000000 - 0.500000000000000*I)*z,
x + (-1.00000000000000*I)*y + (-0.866030000000000 + 0.500000000000000*I)*z,
x + (-1.00000000000000*I)*y + (-0.500000000000000 - 0.866030000000000*I)*z,
x + y + (0.500000000000000 - 0.866030000000000*I)*z,
x + y + (-0.866030000000000 - 0.500000000000000*I)*z,
x + y + (-0.500000000000000 + 0.866030000000000*I)*z,
x + y + (0.866030000000000 + 0.500000000000000*I)*z,
x + (-0.36602540 - 0.36602540*I)*y,
x + (0.36602540 + 0.36602540*I)*y,
x + (-1.3660254 + 1.3660254*I)*y,
x + (1.3660254 - 1.3660254*I)*y]


In [ ]:
investigate_orbit_NEW(QuarticIIIBis[0],GpIII_Dict)

In [ ]:
investigate_isotropy_NEW(QuarticIIIBis[0],GpIII_Dict)

In [ ]:
investigate_orbit_NEW(QuarticIIIBis[27],GpIII_Dict)

In [ ]:
investigate_isotropy_NEW(QuarticIIIBis[27],GpIII_Dict)

# Type IV: The $S_4$ quartic<a name="S4"></a>
We have an $S_4$-symmetric quartic with equation
$$X^4 + Y^4 + Z^4 + a(X^2 Y^2 + Y^2 Z^2 + Z^2 X^2)$$
and symmetries ...

In [ ]:
e_isotropy = []
C2_even = [[24,25,26,27,16,17,18,19,0,1,2,3]]
C2_odd = [[4,5,6,7,14,15,22,23,8,9,10,11]]
S3 = [[12,13,20,21]]
orbit_indices = e_isotropy + C2_even + C2_odd + S3
all_orbit_indices = flatten(orbit_indices)
list_of_bitangent_orbits = []
for o in orbit_indices:
    orbit = []
    for b_index in o:
        orbit.append(S4_Bitangents[b_index])
    list_of_bitangent_orbits.append(orbit)

print('Orbits of bitangents: ' + str(list_of_bitangent_orbits) + '\n')
print('Remaining bitangents:\n')
for j in range(0,len(S4_Bitangents)):
    if not j in all_orbit_indices:
        print(str(j) + ': ' + str(S4_Bitangents[j]))

In [ ]:
investigate_orbit_S4(S4_Bitangents[12],S4_dict)

## (IV): New $S_4$, real points + bitangents

In [ ]:
F=QQ; T.<x,y,z>=PolynomialRing(F)
S4_Quartic_new = x^4 + y^4 + z^4 -3*(x^2*y^2 + y^2*z^2 + z^2*x^2)
S4_Bitangents_new = compute_bitangents(S4_Quartic_new);S4_Bitangents_new
S4_Bitangents_new = [normalize(b) for b in S4_Bitangents_new];S4_Bitangents_new

## $S_4$ dictionary

## IV.3 Orbits

In [ ]:
e_isotropy = []
C2_even = [[0,1,2,3,18,19,26,27,25,24,16,17]]
C2_odd = [[10,4,5,6,11,9,12,20,23,15,8,7]]
S3 = [[13,14,21,22]]
orbit_indices = e_isotropy + C2_even + C2_odd + S3
all_orbit_indices = flatten(orbit_indices)
list_of_bitangent_orbits = []
for o in orbit_indices:
    orbit = []
    for b_index in o:
        orbit.append(S4_Bitangents_new[b_index])
    list_of_bitangent_orbits.append(orbit)

print('Orbits of bitangents: ' + str(list_of_bitangent_orbits) + '\n')
print('Remaining bitangents:\n')
for j in range(0,len(S4_Bitangents_new)):
    if not j in all_orbit_indices:
        print(str(j) + ': ' + str(S4_Bitangents_new[j]))

In [ ]:
investigate_orbit_S4(S4_Bitangents_new[13],S4_dict)

# Type V: The $(16,13)$ quartic<a name="1613"></a>
We have an $G$-symmetric quartic with equation
$$X^4 + Y^4 + Z^4 + aX^2 Y^2$$
This is Type V in Dolgachev. and symmetries ...

## (V) Old

In [ ]:
V_Quartic = x^4 + y^4 + z^4 + x^2*y^2
V_Bitangents = compute_bitangents(V_Quartic)
V_Bitangents = [normalize(b) for b in V_Bitangents]

#############

gen1 = matrix(CC,[[-1,0,0],[0,1,0],[0,0,1]])
gen2 = matrix(CC,[[I,0,0],[0,-I,0],[0,0,1]])
gen3 = matrix(CC,[[0,-1,0],[1,0,0],[0,0,1]])

VGrpElts = [  [ [ 1, 0, 0 ], [ 0, 1, 0 ], [ 0, 0, 1 ] ],
    [ [ -1, 0, 0 ], [ 0, -1, 0 ], [ 0, 0, 1 ] ],
  [ [ -1, 0, 0 ], [ 0, 1, 0 ], [ 0, 0, 1 ] ],
  [ [ 0, -1, 0 ], [ -1, 0, 0 ], [ 0, 0, 1 ] ],
  [ [ 0, -1, 0 ], [ 1, 0, 0 ], [ 0, 0, 1 ] ],
  [ [ 0, 1, 0 ], [ -1, 0, 0 ], [ 0, 0, 1 ] ],
  [ [ 0, 1, 0 ], [ 1, 0, 0 ], [ 0, 0, 1 ] ],
  [ [ 0, -I, 0 ], [ -I, 0, 0 ], [ 0, 0, 1 ] ],
  [ [ 0, -I, 0 ], [ I, 0, 0 ], [ 0, 0, 1 ] ],
  [ [ 0, I, 0 ], [ -I, 0, 0 ], [ 0, 0, 1 ] ],
  [ [ 0, I, 0 ], [ I, 0, 0 ], [ 0, 0, 1 ] ],
  [ [ 1, 0, 0 ], [ 0, -1, 0 ], [ 0, 0, 1 ] ],
  [ [ -I, 0, 0 ], [ 0, -I, 0 ], [ 0, 0, 1 ] ],
  [ [ -I, 0, 0 ], [ 0, I, 0 ], [ 0, 0, 1 ] ],
  [ [ I, 0, 0 ], [ 0, -I, 0 ], [ 0, 0, 1 ] ],
  [ [ I, 0, 0 ], [ 0, I, 0 ], [ 0, 0, 1 ] ] ]

keys = ['r' + str(j) for j in range(0,16)]
VGrp_dict = {}
for j in range(0,16):
    VGrp_dict.update({keys[j]:matrix(CC,VGrpElts[j],immutable=True)});



In [ ]:
###########
someC2 = [[0,1,2,3,20,21,22,23]]
# Stabilized by VGrp_dict['r2']
#      matrix_to_list(VGrp_dict['r2'])

# this is not conjugate to the first,
# stabilized by VGrp_dict['r9']
anotherC2 = [[4,11,12,15,5,13,10,14]]

# stabilized by VGrp_dict['r6'], also not conjugate to the first
yetanotherC2 = [[18,6,16,8,9,17,7,19]]

# the last isotropy is the center of the group, isomorphic to C4
center_C4 = [[24,25,26,27]]

orbit_indices = someC2 + anotherC2 + yetanotherC2 + center_C4

all_orbit_indices = flatten(orbit_indices)
list_of_bitangent_orbits = []
for o in orbit_indices:
    orbit = []
    for b_index in o:
        orbit.append(V_Bitangents[b_index])
    list_of_bitangent_orbits.append(orbit)

print('Orbits of bitangents: ' + str(list_of_bitangent_orbits) + '\n')
print('Remaining bitangents:\n')
for j in range(0,len(V_Bitangents)):
    if not j in all_orbit_indices:
        print(str(j) + ': ' + str(V_Bitangents[j]))

## (V) New curve and bitangents:

In [ ]:
R.<x,y> = PolynomialRing(RR)
maxval=5
V_Quartic_affine = x^4 + y^4 + 1 - 4*x^2*y^2
implicit_plot(V_Quartic_affine,(-maxval,maxval),(-maxval,maxval),color='black')

In [ ]:
F=QQ; T.<x,y,z>=PolynomialRing(F)
V_Quartic_new = x^4 + y^4 + z^4 - 4*x^2*y^2
# V_Quartic_affine = x^4 + y^4 + 1 - 4*x^2*y^2
V_Bitangents_new = compute_bitangents(V_Quartic_new)
V_Bitangents_new = [normalize(b) for b in V_Bitangents_new]

## (V) Orbits of bitangents

In [ ]:
###########
someC2 = [[0,1,2,3,20,21,22,23]]

anotherC2 = [[4,6,17,19,5,7,16,18]]

yetanotherC2 = []

# the last isotropy is the center of the group, isomorphic to C4
center_C4 = [[24,25,26,27]]

orbit_indices = someC2 + anotherC2 + yetanotherC2 + center_C4

all_orbit_indices = flatten(orbit_indices)
list_of_bitangent_orbits = []
for o in orbit_indices:
    orbit = []
    for b_index in o:
        orbit.append(V_Bitangents_new[b_index])
    list_of_bitangent_orbits.append(orbit)

print('Orbits of bitangents: ' + str(list_of_bitangent_orbits) + '\n')
print('Remaining bitangents:\n')
for j in range(0,len(V_Bitangents_new)):
    if not j in all_orbit_indices:
        print(str(j) + ': ' + str(V_Bitangents_new[j]))

In [ ]:
investigate_orbit(V_Bitangents_new[0],VGrp_dict)

TODO --- find the isotropy for this orbit

# Type VI: The $C_9$ quartic<a name="C9"></a>
We have an $C_9$-symmetric quartic with equation
$$X^4 + XY^3 + YZ^3$$
and symmetries ...

In [ ]:
C9_Quartic = x^4 + x*y^3 + y*z^3
C9_Bitangents = compute_bitangents(C9_Quartic)
C9_Bitangents = [normalize(bitangent) for bitangent in C9_Bitangents];C9_Bitangents

In [ ]:
e_isotropy = [[4,0,23,24,25,3,26,8,7],[6,1,5,20,12,21,22,19,11],[2,9,10,13,14,15,16,17,18]]
C3_isotropy = []
C9_isotropy = [[27]]
orbit_indices = e_isotropy + C3_isotropy + C9_isotropy
all_orbit_indices = flatten(orbit_indices)
list_of_bitangent_orbits = []
for o in orbit_indices:
    orbit = []
    for b_index in o:
        orbit.append(C9_Bitangents[b_index])
    list_of_bitangent_orbits.append(orbit)

# remaining_bitangents = rescaled_C3_Bitangents

print('Orbits of bitangents: ' + str(list_of_bitangent_orbits) + '\n')
print('Remaining bitangents:\n')
for j in range(0,len(C9_Bitangents)):
    if not j in all_orbit_indices:
        print(str(j) + ': ' + str(C9_Bitangents[j]))

In [ ]:
investigate_orbit(C9_Bitangents[27],C9_dict)

# Type VII: The $D_4$ quartic<a name="D4"></a>
We have an $D_4$-symmetric quartic with equation
$$X^4 + Y^4 + Z^4 + aX^2 Y^2 + bXYZ^2$$
and symmetries ...

## (VII) Old:

In [ ]:
D4_Quartic = x^4 + y^4 + z^4 + x^2*y^2 + x*y*z^2
D4_Bitangents = compute_bitangents(D4_Quartic)
D4_Bitangents = [normalize(bitangent) for bitangent in D4_Bitangents]

In [ ]:
# Ordered by isotropy, these are all transitive D8-sets
trivial = [[16,17,11,20,10,6,21,7]]
centerC2 = [[24,25,26,27]]
otherC2 = [[0,12,3,13],[1,2,14,15],[4,9,18,23],[5,8,19,22]]
K4 = []
C4 = []
D8 = []

orbit_indices = trivial + centerC2 + otherC2 + K4 + C4 + D8
all_orbit_indices = flatten(orbit_indices)
list_of_bitangent_orbits = []
for o in orbit_indices:
    orbit = []
    for b_index in o:
        orbit.append(D4_Bitangents[b_index])
    list_of_bitangent_orbits.append(orbit)

# remaining_bitangents = rescaled_C3_Bitangents

print('Orbits of bitangents: ' + str(list_of_bitangent_orbits) + '\n')
print('Remaining bitangents:\n')
for j in range(0,len(D4_Bitangents)):
    if not j in all_orbit_indices:
        print(str(j) + ': ' + str(D4_Bitangents[j]))

So we get, where $C_2^Z \le D_8$ denotes the center subgroup, and $C_2$ without a decoration denotes the non-center conjugacy subgroup which is cyclic of order two:
$$[D_8/e] + 4[D_8/C_2] + [D_8/C_2^Z]$$

## (VII) New equation

In [ ]:
R.<x,y> = PolynomialRing(RR)
maxval=5
D4_Quartic_affine = x^4 + y^4 + 1 - 3*x^2*y^2 + x*y
implicit_plot(D4_Quartic_affine,(-maxval,maxval),(-maxval,maxval),color='black')

In [ ]:
F=QQ; T.<x,y,z>=PolynomialRing(F)
D4_Quartic_new = x^4 + y^4 + z^4 - 3*x^2*y^2 + x*y*z^2
D4_Bitangents_new = compute_bitangents(D4_Quartic_new)
D4_Bitangents_new = [normalize(bitangent) for bitangent in D4_Bitangents_new];D4_Bitangents_new

## (VII) Dict

## (VII) Orbits

In [ ]:
# Ordered by isotropy, these are all transitive D8-sets
trivial = [[2,1,13,12,0,3,14,15]]
centerC2 = [[24,25,26,27]]
otherC2 = [[4,7,20,23],[22,6,5,21]] # iso4 in GAP
yet_another_C2 = [[19,9,8,18],[10,16,17,11]] # iso8 in GAP

orbit_indices = trivial + centerC2 + otherC2 + yet_another_C2
all_orbit_indices = flatten(orbit_indices)
list_of_bitangent_orbits = []
for o in orbit_indices:
    orbit = []
    for b_index in o:
        orbit.append(D4_Bitangents_new[b_index])
    list_of_bitangent_orbits.append(orbit)

# remaining_bitangents = rescaled_C3_Bitangents

print('Orbits of bitangents: ' + str(list_of_bitangent_orbits) + '\n')
print('Remaining bitangents:\n')
for j in range(0,len(D4_Bitangents_new)):
    if not j in all_orbit_indices:
        print(str(j) + ': ' + str(D4_Bitangents_new[j]))

In [ ]:
investigate_orbit(D4_Bitangents_new[24],D8_dict)

# Type VIII: $C_6$<a name="C6"></a>
Doing a new equation the second time around:

In [ ]:
F=QQ; T.<x,y,z>=PolynomialRing(F)
C6_Quartic = z^3*y +x^4 -3*x^2*y^2 +y^4
C6_Bitangents = compute_bitangents(C6_Quartic);C6_Bitangents
C6_Bitangents = [normalize(bitangent) for bitangent in C6_Bitangents];C6_Bitangents

There are some rounding errors so we edit this set by hand:

In [ ]:
C6_Bitangents=[y + (-0.92831777)*z,
 y + (0.46415888 + 0.80394677*I)*z,
 y + (0.46415888 - 0.80394677*I)*z,
 x + 1.85047*y + (-0.39283)*z,
 x + (-1.85047)*y + 0.39283*z,
 x + 0.56125*I*y + 1.18243*I*z,
 x + (-0.56125*I)*y + (-1.18243*I)*z,
 x + (0.70756 - 0.23507*I)*y + (-0.52306 - 0.12198*I)*z,
 x + (-0.70756 + 0.23507*I)*y + (0.52306 + 0.12198*I)*z,
 x + (0.70756 + 0.23507*I)*y + (-0.52306 + 0.12198*I)*z,
 x + (-0.70756 - 0.23507*I)*y + (0.52306 - 0.12198*I)*z,
 x + (-0.56125*I)*y + (-1.02401 + 0.59121*I)*z,
 x + 0.56125*I*y + (1.02401 - 0.59121*I)*z,
 x + 0.56125*I*y + (-1.02401 - 0.59121*I)*z,
 x + (-0.56125*I)*y + (1.02401 + 0.59121*I)*z,
 x + (-0.70756 - 0.23507*I)*y + (-0.15589 + 0.51397*I)*z,
 x + (0.70756 + 0.23507*I)*y + (0.15589 - 0.51397*I)*z,
 x + (-0.70756 + 0.23507*I)*y + (-0.15589 - 0.51397*I)*z,
 x + (0.70756 - 0.23507*I)*y + (0.15589 + 0.51397*I)*z,
 x + (-0.70756 + 0.23507*I)*y + (-0.36717 + 0.39199*I)*z,
 x + (0.70756 - 0.23507*I)*y + (0.36717 - 0.39199*I)*z,
 x + (-0.70756 - 0.23507*I)*y + (-0.36717 - 0.39199*I)*z,
 x + (0.70756 + 0.23507*I)*y + (0.36717 + 0.39199*I)*z,
 x + (-1.85047)*y + (-0.19641 - 0.3402*I)*z,
 x + 1.85047*y + (0.19641 + 0.3402*I)*z,
 x + (-1.85047)*y + (-0.19641 + 0.3402*I)*z,
 x + 1.85047*y + (0.19641 - 0.3402*I)*z,
 y]

In [ ]:
# C6 = <r> so the elements are ordered so that dict['ri'] = r^i
C6_elts = [
    [[1,0,0],[0,1,0],[0,0,1]],
    [ [ -1, 0, 0 ], [ 0, 1, 0 ], [ 0, 0, E(3) ] ],
    [ [ 1, 0, 0 ], [ 0, 1, 0 ], [ 0, 0, E(3)^2 ] ],
    [ [ -1, 0, 0 ], [ 0, 1, 0 ], [ 0, 0, 1 ] ],
    [ [ 1, 0, 0 ], [ 0, 1, 0 ], [ 0, 0, E(3) ] ],
    [ [ -1, 0, 0 ], [ 0, 1, 0 ], [ 0, 0, E(3)^2 ] ]
]


keys = ['r' + str(j) for j in range(0,6)]
C6_dict = {}
for j in range(0,6):
    C6_dict.update({keys[j]:matrix(CC,C6_elts[j],immutable=True)});

## (VIII) Orbits:

In [ ]:
e_isotropy = [[24,26,3,25,4,23],[6,11,14,13,12,5],[7,20,18,19,8,17],[16,9,10,22,15,21]]
C2_isotropy = [[0,1,2]]

orbit_indices = e_isotropy + C2_isotropy
all_orbit_indices = flatten(orbit_indices)
list_of_bitangent_orbits = []
for o in orbit_indices:
    orbit = []
    for b_index in o:
        orbit.append(C6_Bitangents[b_index])
    list_of_bitangent_orbits.append(orbit)

print('Orbits of bitangents: ' + str(list_of_bitangent_orbits) + '\n')
print('Remaining bitangents:\n')
for j in range(0,len(C6_Bitangents)):
    if not j in all_orbit_indices:
        print(str(j) + ': ' + str(C6_Bitangents[j]))

In [ ]:
investigate_orbit(C6_Bitangents[27],C6_dict)

# Type IX: The $S_3$ quartic<a name="S3"></a>
We have an $S_3$-symmetric quartic with equation
$$X^3Z + Y^3 Z + X^2 Y^2 + aXYZ^2 + bZ^4$$
and symmetries ...

## (IX) Old

In [ ]:
S3_Quartic = x^3 * z + y^3 * z + x^2 * y^2 - x*y*z^2 + z^4
S3_Bitangents = compute_bitangents(S3_Quartic)
S3_Bitangents = [normalize(bitangent) for bitangent in S3_Bitangents]

rho = exp(2*pi*i/3)

cycle1 = matrix(CC,[[rho,0,0],[0,rho^2,0],[0,0,1]])
transp1 = matrix([[0,1,0],[1,0,0],[0,0,1]])
transp2 = cycle1*transp1
transp3 = cycle1*cycle1*transp1
cycle2 = cycle1*cycle1

In [ ]:
orbit1 = [[2]]
orbit2 = []
orbit3 = [[0,26,27],[1,18,19],[3,8,9]]
orbit6 = [[4,5,16,23,22,17],[6,14,21,13,24,11],[7,15,12,20,10,25]]
orbit_indices = orbit1 + orbit2 + orbit3 + orbit6
all_orbit_indices = flatten(orbit_indices)
# Get the running list of orbits that we have
list_of_bitangent_orbits = []
for o in orbit_indices:
    orbit = []
    for b_index in o:
        orbit.append(S3_Bitangents[b_index])
    list_of_bitangent_orbits.append(orbit)

# remaining_bitangents = rescaled_C3_Bitangents

# print('Orbits of bitangents: ' + str(list_of_bitangent_orbits) + '\n')
# print('Remaining bitangents:\n')
# for j in range(0,len(S3_Bitangents)):
#     if not j in all_orbit_indices:
#         print(str(j) + ': ' + str(S3_Bitangents[j]))

So we get orbits:
$$[S_3/S_3] + 3[S_3/C_2] + 3[S_3/e]$$

## (IX) New

In [ ]:
# S3_Quartic_new = x^3 * z + y^3 * z + x^2 * y^2 - 25*x*y*z^2 + 10*z^4
R.<x,y> = PolynomialRing(RR)
maxval=10
S3_Quartic_new_affine = x^3 + y^3 + x^2 * y^2 - 25*x*y + 10
implicit_plot(S3_Quartic_new_affine,(-maxval,maxval),(-maxval,maxval),color='black')

In [ ]:
F=QQ; T.<x,y,z>=PolynomialRing(F)
S3_Quartic_new = x^3 * z + y^3 * z + x^2 * y^2 - 25*x*y*z^2 + 10*z^4
S3_Bitangents = compute_bitangents(S3_Quartic_new)
S3_Bitangents = [normalize(bitangent) for bitangent in S3_Bitangents];S3_Bitangents

## (IX) Dictionary

In [ ]:
S3_elts=[ [ [ 0, 1, 0 ], [ 1, 0, 0 ], [ 0, 0, 1 ] ],
  [ [ 0, E(3)^2, 0 ], [ E(3), 0, 0 ], [ 0, 0, 1 ] ],
  [ [ 0, E(3), 0 ], [ E(3)^2, 0, 0 ], [ 0, 0, 1 ] ],
  [ [ 1, 0, 0 ], [ 0, 1, 0 ], [ 0, 0, 1 ] ],
  [ [ E(3)^2, 0, 0 ], [ 0, E(3), 0 ], [ 0, 0, 1 ] ],
  [ [ E(3), 0, 0 ], [ 0, E(3)^2, 0 ], [ 0, 0, 1 ] ] ];

keys = ['r' + str(j) for j in range(0,6)]
S3_dict = {}
for j in range(0,6):
    S3_dict.update({keys[j]:matrix(CC,S3_elts[j],immutable=True)});

## (IX) Orbits

In [ ]:
triv_orbit = [[1,23,22,26,27,0],[14,21,15,20,2,4],[6,10,11,8,7,9]]
C2_orbit = [[18,19,3],[12,24,17],[16,25,13]]
S3_orbit = [[5]]

orbit_indices = triv_orbit + C2_orbit
all_orbit_indices = flatten(orbit_indices)
list_of_bitangent_orbits = []
for o in orbit_indices:
    orbit = []
    for b_index in o:
        orbit.append(C6_Bitangents[b_index])
    list_of_bitangent_orbits.append(orbit)

print('Orbits of bitangents: ' + str(list_of_bitangent_orbits) + '\n')
print('Remaining bitangents:\n')
for j in range(0,len(S3_Bitangents)):
    if not j in all_orbit_indices:
        print(str(j) + ': ' + str(S3_Bitangents[j]))

In [ ]:
investigate_orbit(S3_Bitangents[13],S3_dict)

# Type X: The $K_4$ quartic<a name="K4"></a>
We now look at a quartic with an action by the Klein four group, given by the equation $$X^4 + Y^4 + Z^4 + aX^2 Y^2 + bY^2Z^2 + c Z^2 X^2$$ with action given by $$\text{diag}[-1,1,1] \qquad \text{and}\qquad \text{diag}[1,-1,1]$$

## Old

In [ ]:
K4_Quartic = x^4 + y^4 + z^4 + x^2*y^2 + y^2*z^2 + z^2*x^2
K4_Bitangents = compute_bitangents(K4_Quartic)
K4_Bitangents = [normalize(bitangent) for bitangent in K4_Bitangents]

# There are three subgroups of K4, the left, middle, and diagonal copies of C2
left_generator = matrix([[-1,0,0],[0,1,0],[0,0,1]])
right_generator = matrix([[1,0,0],[0,-1,0],[0,0,1]])
diagonal_generator = matrix([[-1,0,0],[0,-1,0],[0,0,1]])

# Make different lists to indicate different isotropy subgroups
left_isotropy_indices = [[0,1],[2,3]]
right_isotropy_indices = [[16,18],[17,19]]
diagonal_isotropy_indices = [[24,26],[25,27]]
remaining_orbits = [[12,13,20,21],[4,5,6,7],[8,9,10,11],[14,15,22,23]]
orbit_indices = left_isotropy_indices + right_isotropy_indices + diagonal_isotropy_indices + remaining_orbits
# Flatten the union
all_orbit_indices = flatten(orbit_indices)

# Get the running list of orbits that we have
list_of_bitangent_orbits = []
for o in orbit_indices:
    orbit = []
    for b_index in o:
        orbit.append(K4_Bitangents[b_index])
    list_of_bitangent_orbits.append(orbit)

# remaining_bitangents = rescaled_C3_Bitangents

print('Orbits of bitangents: ' + str(list_of_bitangent_orbits) + '\n')
print('Remaining bitangents:\n')
for j in range(0,len(K4_Bitangents)):
    if not j in all_orbit_indices:
        print(str(j) + ': ' + str(K4_Bitangents[j]))

In [ ]:
candidate_orbit(K4_Bitangents[25],[left_generator,right_generator,diagonal_generator])

So the orbits for bitangents on our $K_4$-equivariant quartic have the form
$$2[K_4/C_2^L] + 2[K_4/C_2^\Delta] + 2[K_4/C_2^R] + 4[K_4/e]$$

## (X) New

In [ ]:
F=QQ; T.<x,y,z>=PolynomialRing(F)

K4_Quartic_new = x^4 + y^4 + z^4 -9*x^2*y^2 - 3*y^2*z^2 - 8*x^2*z^2

K4_Bitangents_new = compute_bitangents(K4_Quartic_new)
K4_Bitangents_new = [normalize(bitangent) for bitangent in K4_Bitangents_new];K4_Bitangents_new

## (X) Dictionary

In [ ]:
K4_dict = {}
K4_dict.update({'id':matrix(CC,[[1,0,0],[0,1,0],[0,0,1]],immutable=True)})
K4_dict.update({'leftgen':matrix(CC,[[-1,0,0],[0,1,0],[0,0,1]],immutable=True)})
K4_dict.update({'rightgen':matrix(CC,[[1,0,0],[0,-1,0],[0,0,1]],immutable=True)})
K4_dict.update({'diaggen':matrix(CC,[[-1,0,0],[0,-1,0],[0,0,1]],immutable=True)})

## (X) Orbits

In [ ]:
# e_orb = [[4,5,18,19],[6,7,16,17],[8,9,14,15],[10,11,12,13],[20,21,24,25]]
# C2L = [[0,1],[2,3]]
# C2R = [[22,23],[26,27]]

orbit_indices = []
# Flatten the union
all_orbit_indices = flatten(orbit_indices)

# Get the running list of orbits that we have
list_of_bitangent_orbits = []
for o in orbit_indices:
    orbit = []
    for b_index in o:
        orbit.append(K4_Bitangents_new[b_index])
    list_of_bitangent_orbits.append(orbit)

print('Orbits of bitangents: ' + str(list_of_bitangent_orbits) + '\n')
print('Remaining bitangents:\n')
for j in range(0,len(K4_Bitangents_new)):
    if not j in all_orbit_indices:
        print(str(j) + ': ' + str(K4_Bitangents_new[j]))

In [ ]:
investigate_orbit(K4_Bitangents_new[24],K4_dict)

# Type XI: The $C_3$ quartic<a name="C3"></a>
The first one we're trying is the one with automorphism group $C_3$ which we will call `C3_Quartic`. The associated list of bitangents is `C3_Bitangents`.

In [ ]:
C3_Quartic = z^3*y + x*(x-y)*(x-2*y)*(x-3*y)
C3_Bitangents = compute_bitangents(C3_Quartic)
rescaled_C3_Bitangents = [normalize(bitangent) for bitangent in C3_Bitangents]
# for j in range(0,len(rescaled_C3_Bitangents)):
#     print(str(j) + ': ' + str(rescaled_C3_Bitangents[j]))

In [ ]:
# The action generating the C3 orbit
rho = exp(2*pi*i/3)
M = matrix([[1,0,0],[0,1,0],[0,0,rho]])

orbit_indices = [[1,15,16],[2,11,12],[5,14,19],[0,25,26],[3,18,23],[27],[4,17,24],[6,13,20],[7,10,21],[8,9,22]]
all_orbit_indices = flatten(orbit_indices)

# Get the running list of orbits that we have
list_of_bitangent_orbits = []
for o in orbit_indices:
    orbit = []
    for b_index in o:
        orbit.append(rescaled_C3_Bitangents[b_index])
    list_of_bitangent_orbits.append(orbit)

# remaining_bitangents = rescaled_C3_Bitangents

# gp_action(M,rescaled_C3_Bitangents[22])
print('Orbits of bitangents: ' + str(list_of_bitangent_orbits) + '\n')
print('Remaining bitangents:\n')
for j in range(0,len(rescaled_C3_Bitangents)):
    if not j in all_orbit_indices:
        print(str(j) + ': ' + str(rescaled_C3_Bitangents[j]))

Okay so now we know the orbits of the $C_3$-equivariant quartic: $9[C_3/e]+[C_3/C_3]$

# Type XII: The $C_2$ quartic

## (XII) Old

In [ ]:
L2 = y^2 - z^2
L4 = y^4 - y^2*z^2 + z^4
C2_Quartic = x^4 + x^2*L2 + L4

C2_Bitangents = compute_bitangents(C2_Quartic);
C2_Bitangents = [normalize(b) for b in C2_Bitangents]

e_orbits = [[0],[1],[2],[3]]
C2_orbits = [[4,5],[6,7],[8,9],[10,11],[12,14],[13,15],[16,17],[18,19],[20,21],[22,23],[24,26],[25,27]]

C2_Bitangents

In [ ]:
m = matrix(CC,[[-1,0,0],[0,1,0],[0,0,1]])
print(C2_Bitangents[4])
gp_action(m,C2_Bitangents[4])

## (XII) New

In [ ]:
L2_new = -y^2 - 4*z^2
L4_new = y^4 - y^2*z^2 - z^4
C2_Quartic_new = x^4 + x^2*L2_new + L4_new

C2_Bitangents_new = compute_bitangents(C2_Quartic_new);
C2_Bitangents_new = [normalize(b) for b in C2_Bitangents_new]

## (XII) Orbits
The action is given by the diagonal matrix $\begin{pmatrix} -1 & 0 & 0 \\ 0 & 1 & 0 \\ 0 & 0 & 1\end{pmatrix}$ so it just flips the sign on the $x$ (or on both $y$ and $z$), so we can eyeball the orbits. Note if the coefficient on $x$ is trivial then the action doesn't affect the bitangent.

In [ ]:
e_orbs = []
# [[0],[1],[2],[3]]
C2_orbs = []
# [[4,5],[6,7],[8,9],[10,11],[12,13],[14,15],[16,17],..]
orbit_indices = e_orbs + C2_orbs
all_orbit_indices = flatten(orbit_indices)

# Get the running list of orbits that we have
list_of_bitangent_orbits = []
for o in orbit_indices:
    orbit = []
    for b_index in o:
        orbit.append(C2_Bitangents_new[b_index])
    list_of_bitangent_orbits.append(orbit)

print('Orbits of bitangents: ' + str(list_of_bitangent_orbits) + '\n')
print('Remaining bitangents:\n')
for j in range(0,len(C2_Bitangents_new)):
    if not j in all_orbit_indices:
        print(str(j) + ': ' + str(C2_Bitangents_new[j]))

The action of $m$ leaves the bitangent stable if there's no $x$ coefficient, otherwise it swaps the sign on the $y$ and $z$ coefficients.

In [ ]:
2+2

# Trott / Blum-Guinand

In [ ]:
Trott = 144*(x^4+y^4) - 255*(x^2+y^2)*z^2 + 350*x^2*y^2+81*z^4
Trott_bis = compute_bitangents(Trott);Trott_bis


*Weird*: Why are some of them complex?

# Edge quartic

In [ ]:
Edge = 25*(x^4 + y^4 + z^4) -34*(x^2*y^2 + x^2*z^2 + y^2*z^2)
Edge_bis = compute_bitangents(Edge);
Edge_bis = [normalize(b) for b in Edge_bis];Edge_bis

In [ ]:
c2r1 = [[16,19,0,3],[1,18,2,17]]
c2r5 = [[11,10,5,4],[20,15,23,12]]
e_istrop = [[21,13,7,9,14,22,6,8]]
c2_r0 = [[24,25,26,27]]

orbit_indices = c2r1 + c2r5 + e_istrop

all_orbit_indices = flatten(orbit_indices)

# Get the running list of orbits that we have
list_of_bitangent_orbits = []
for o in orbit_indices:
    orbit = []
    for b_index in o:
        orbit.append(Edge_bis[b_index])
    list_of_bitangent_orbits.append(orbit)

print('Orbits of bitangents: ' + str(list_of_bitangent_orbits) + '\n')
print('Remaining bitangents:\n')
for j in range(0,len(Edge_bis)):
    if not j in all_orbit_indices:
        print(str(j) + ': ' + str(Edge_bis[j]))

## Edge with $S_4$

In [ ]:
TypeIV_S4 = [ [ [ -1, 0, 0 ], [ 0, -1, 0 ], [ 0, 0, 1 ] ],
  [ [ -1, 0, 0 ], [ 0, 0, -1 ], [ 0, -1, 0 ] ],
  [ [ -1, 0, 0 ], [ 0, 0, 1 ], [ 0, 1, 0 ] ],
  [ [ -1, 0, 0 ], [ 0, 1, 0 ], [ 0, 0, -1 ] ],
  [ [ 0, -1, 0 ], [ -1, 0, 0 ], [ 0, 0, -1 ] ],
  [ [ 0, -1, 0 ], [ 0, 0, -1 ], [ 1, 0, 0 ] ],
  [ [ 0, -1, 0 ], [ 0, 0, 1 ], [ -1, 0, 0 ] ],
  [ [ 0, -1, 0 ], [ 1, 0, 0 ], [ 0, 0, 1 ] ],
  [ [ 0, 0, -1 ], [ -1, 0, 0 ], [ 0, 1, 0 ] ],
  [ [ 0, 0, -1 ], [ 0, -1, 0 ], [ -1, 0, 0 ] ],
  [ [ 0, 0, -1 ], [ 0, 1, 0 ], [ 1, 0, 0 ] ],
  [ [ 0, 0, -1 ], [ 1, 0, 0 ], [ 0, -1, 0 ] ],
  [ [ 0, 0, 1 ], [ -1, 0, 0 ], [ 0, -1, 0 ] ],
  [ [ 0, 0, 1 ], [ 0, -1, 0 ], [ 1, 0, 0 ] ],
  [ [ 0, 0, 1 ], [ 0, 1, 0 ], [ -1, 0, 0 ] ],
  [ [ 0, 0, 1 ], [ 1, 0, 0 ], [ 0, 1, 0 ] ],
  [ [ 0, 1, 0 ], [ -1, 0, 0 ], [ 0, 0, 1 ] ],
  [ [ 0, 1, 0 ], [ 0, 0, -1 ], [ -1, 0, 0 ] ],
  [ [ 0, 1, 0 ], [ 0, 0, 1 ], [ 1, 0, 0 ] ],
  [ [ 0, 1, 0 ], [ 1, 0, 0 ], [ 0, 0, -1 ] ],
  [ [ 1, 0, 0 ], [ 0, -1, 0 ], [ 0, 0, -1 ] ],
  [ [ 1, 0, 0 ], [ 0, 0, -1 ], [ 0, 1, 0 ] ],
  [ [ 1, 0, 0 ], [ 0, 0, 1 ], [ 0, -1, 0 ] ],
  [ [ 1, 0, 0 ], [ 0, 1, 0 ], [ 0, 0, 1 ] ] ]
Edge_S4_dict = {}
keys = ['r' + str(j) for j in range(0,24)]
for j in range(0,24):
    Edge_S4_dict.update({keys[j]:matrix(CC,TypeIV_S4[j],immutable=True)});

In [ ]:
c2r20 = [[18,25,24,27,16,19,26,17,0,1,2,3]]
c2r4 = [[21,14,13,22,11,4,10,5,7,8,9,6]]

orbit_indices = c2r20 + c2r4

all_orbit_indices = flatten(orbit_indices)

# Get the running list of orbits that we have
list_of_bitangent_orbits = []
for o in orbit_indices:
    orbit = []
    for b_index in o:
        orbit.append(Edge_bis[b_index])
    list_of_bitangent_orbits.append(orbit)

print('Orbits of bitangents: ' + str(list_of_bitangent_orbits) + '\n')
print('Remaining bitangents:\n')
for j in range(0,len(Edge_bis)):
    if not j in all_orbit_indices:
        print(str(j) + ': ' + str(Edge_bis[j]))

In [ ]:
investigate_orbit(Edge_bis[12],Edge_S4_dict)

In [ ]:
Edge_S4_dict['r4']